In [24]:
from __future__ import division,print_function,absolute_import 
import collections
import os
import random
import urllib
import zipfile
import numpy as np
import tensorflow as tf

In [25]:
learning_rate=0.1
bstch_size=128
num_steps=1000000
display_step=10000
eval_step=200000#???

eval_words=['five','of','going','hardware','american','britain']

embedding_size=200
max_vocabulary_size=50000
min_occurrence=10
skip_window=3
num_skips=2
num_sampled=64#负样本数目

In [ ]:
# url='http://mattmahoney.net/dc/text8.zip'
# data_path='text8.zip'
# if not os.path.exists(data_path):
#     print("Downloading...")
#     filename, _ = urllib.urlretrieve(url, data_path)
# #     filename,_=urllib.urlretrieve(url,data_path)
#     print("Done")
# #解压
# with zipfile.ZipFile(data_path) as f:
#     text_words=f.read(f.namelist()[0]).lower().split()

In [26]:
# Download a small chunk of Wikipedia articles collection
url = 'http://mattmahoney.net/dc/text8.zip'
data_path = 'text8.zip'
if not os.path.exists(data_path):
    print("Downloading the dataset... (It may take some time)")
    filename, _ = urllib.request.urlretrieve(url, data_path)
    print("Done!")
# Unzip the dataset file. Text has already been processed
with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()
#text_words 是单词集合，一堆语料去掉段落和标点符号

In [29]:
count=[('UNK',-1)]#只有一组数据
#max_common_size确定了字典的容量，取text_words(是字典)语料里最常见的max_vocabulary_size个，
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size-1))
print(count)
for i in range(len(count)-1,-1):
    if count[i][1]<min_occurrence:
        count.pop(i)
    else:
        break
vocabulary_size=len(count)

#计算单词在count的位置
word2id=dict()#先产生一个空字典

for i, (word,_) in enumerate(count):
    word2id[word]=i
    print(i)
    print(word2id)
    
#将单词转成单词的id
data=list()
unk_count= 0

for word in text_words:
    index = word2id.get(word,0)
    if index ==0:
        unk_count+=1
    data.append(index)
#更新低频词汇的数量
count[0]=('UNK',unk_count)
#id和单词的对应关系
id2word=dict(zip(word2id.values(),word2id.keys()))

print("Words count:",len(text_words))
print("Unique words:",len(set(text_words)))
print("Vocabulary size:",vocabulary_size)
print("Most common words:",count[:10])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Words count: 17005207
Unique words: 253854
Vocabulary size: 50000
Most common words: [('UNK', 418391), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [30]:
data_index=0
#生成 skip-germ模型
def next_batch(batch_size,num_skips,skip_window):
    global data_index
    assert batch_size % num_skips==0
    assert num_skips <=2 * skip_window 
    batch=np.ndarray(shape=(batch_size),dtype=np.int32)
    labels=np.ndarray(shape=(batch_size),dtypw=np.int32)
    #获取window 大小
    span=2*skip_window +1
    buffer=collections.deque(maxlen=span)
    if data_index+span>len(data):
        data_index=0
    buffer.dextend(data[data_index:data_index+span])
    data_index+=span
    for i in range(batch_size/num_skips):
        context_words=[w for w in range(span) if w!= skip_window]
        words_to_use=random.sample(context_words,num_skips)
        for j,context_words in enumerate(words_to_use):
            batch[i*num_skips+j]=buffer[skip_window]
            labels[i*num_skips+j,0]=buffer[context_word]
        if data_index ==len(data):
            buffer.extend(data[0:span])
            data_index=span
        else:
            buffer.append(data_index)
            data_index+=1
    #通过回溯的方法，保证每个batch最后的不被略过
    data_index=(data_index+len(data)-span)% len(data)
    return batch,labels